In [3]:
import enum
import copy
import numbers
import itertools
import functools
import collections
import operator
import json
import logging

import utility as util

In [8]:
for t in itertools.count():
    print(t)
    if t >= 5:
        break

0
1
2
3
4
5


In [7]:
# Check whether a value is a number
isinstance(5, numbers.Number), isinstance(5.1, numbers.Number), isinstance('5.1', numbers.Number)

(True, True, False)

In [10]:
# TODO: isn't there a way to include exception codes?
try:
    raise Exception("Something went wrong.")
except Exception as e:
    print(e)
    print(repr(e))

Something went wrong.
Exception('Something went wrong.')


In [7]:
# Currying functions using functools.partial()
def myfunc(a, b, c, d=0):
    print(a, b, c, d)
_myfunc = functools.partial(myfunc, 1, 2, d=4)
_myfunc(3)
_myfunc = functools.partial(myfunc, 1)
_myfunc(2, 3)

1 2 3 4
1 2 3 0


In [10]:
A = collections.namedtuple('A', ('c', 'd',))
a = A(1, 2)
print(a.c, a.d)
a1 = A(1, 2)
a2 = A(3, 4)
a3 = A(5, 6)

batch = A(*zip(*[a1, a2, a3]))
print(batch.c, batch.d)

1 2
(1, 3, 5) (2, 4, 6)


Iterators and iterable.

What exactly are iterator, iterable, and iteration? [StackOverflow](https://stackoverflow.com/questions/9884132/what-exactly-are-iterator-iterable-and-iteration)

In [ ]:
class Itor:
    """Iterator"""
    def __init__(self):
        self.__i = -1
        
    def __iter__(self):
        self.__i = -1
        return self
    
    def __next__(self):
        self.__i += 1
        return self.__i
    
class Itble:
    """Iterable"""
    def __init__(self, l):
        self.__l = l
    
    def __iter__(self):
        for i in self.__l:
            yield i
        
it1 = Itor()
it2 = Itble([1, 2, 3, 4, 5])

[next(it1) for _ in range(6)], list(it2)

Queues

Use `collections.deque` and not `queue.Queue`.

In [8]:
q = collections.deque([1, 2, 3])
q.append(4)
q.extend([5,6])
l = []
while q:
    i = q.pop()
    l.append(i)
l

[6, 5, 4, 3, 2, 1]

In [11]:
# Joining and splitting strings
l = ['a', 'b', 'c']
s = '/'.join(l)
s, s.split('/')

('a/b/c', ['a', 'b', 'c'])

In [9]:
# Sorting iterables
l = [1,3,2]
s = {1,3,2}
r = range(3, 0, -1)
sorted(l), sorted(s), sorted(r)

([1, 2, 3], [1, 2, 3], [1, 2, 3])

In [8]:
# Filter items in a dict by inverse mapping
d = {'a': 1, 'b': 1, 'c': 2, 'd': 3, 'e': 3}
{v: k for k, v in d.items()}

{1: 'b', 2: 'c', 3: 'e'}

In [3]:
# Using zip(), and easy way to create dict() object
zip_to_list = lambda *args: list(zip(*args))

zip_to_list('abc', [1, 2, 3])

([('a', 1), ('b', 2), ('c', 3)],
 {'a': 1, 'b': 2, 'c': 3},
 OrderedDict([('a', 1), ('b', 2), ('c', 3)]))

In [2]:
# Using dict
def subdict(d, ks):
    return {k: d[k] for k in ks if k in d}

def map_to_list(f, l):
    return list(map(f, l))

def from_dict_with_remainder(d, ks):
    leftover = set(ks) - set(d.keys())
    ks = set(ks) & set(d.keys())
    return map_to_list(d.get, ks), list(leftover)

d = dict(zip('abcd', range(4)))

d, subdict(d, 'abe'), subdict({}, 'abe'), map_to_list(d.get, 'abe'), from_dict_with_remainder(d, 'cdef')

({'a': 0, 'b': 1, 'c': 2, 'd': 3},
 {'a': 0, 'b': 1},
 {},
 [0, 1, None],
 ([2, 3], ['f', 'e']))

Chaining lists.

What is the difference between chain and chain.from_iterable in itertools? [StackOverflow](https://stackoverflow.com/questions/15004772/what-is-the-difference-between-chain-and-chain-from-iterable-in-itertools)

In [3]:
l = [[1,2,3], [4,5], [6]]

list( itertools.chain(*l) ), list( itertools.chain.from_iterable(l) )

([1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6])

In [24]:
# Using accumulate()
accumulate_to_list = lambda *args, **kwargs: list(itertools.accumulate(*args, **kwargs))
class A:
    def __init__(self, a):
        self.a = a
    def __repr__(self):
        return f"a={self.a}"
v = [A(2), A(3), A(4), A(5)]
add = lambda c,d: A(c.a + d.a)

accumulate_to_list(v, add), accumulate_to_list(v, add, initial=A(10))

([a=1, a=3, a=6, a=10, a=15], [a=10, a=11, a=13, a=16, a=20, a=25])

In [41]:
# Using tee() to make a list of consecutive lists
# itertools.tee() returns n independent iterators from a single iterable.

def pairwise_to_list(l):
    a, b = itertools.tee(l)
    next(b, None)
    return list(zip(a, b))

pairwise_to_list(range(5)), pairwise_to_list(range(2)), pairwise_to_list(range(1))

([(0, 1), (1, 2), (2, 3), (3, 4)], [(0, 1)], [])

In [38]:
def pairwise_do(f, l):
    a, b = itertools.tee(l)
    next(b, None)
    return [f(i, j) for i, j in zip(a, b)]
class A:
    def __init__(self, a):
        self.a = a
    def __repr__(self):
        return f"a={self.a}"
v = [A(1), A(2), A(3), A(4), A(5)]
add = lambda c,d: A(c.a + d.a)

pairwise_do(add, v)

[a=3, a=5, a=7, a=9]

In [23]:
# Using reduce()
v = ['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
functools.reduce(lambda acc, x: acc + ' ' + x, v), functools.reduce(lambda acc, x: acc + ' ' + x, v, 'LOG:')

('The quick brown fox jumps over the lazy dog',
 'LOG: The quick brown fox jumps over the lazy dog')

In [17]:
# Getting names of classes. Works with primitives
class A:
    pass
a = A()
i = 1
s = 'x'
def classname_1(x):
    return x.__class__.__name__
def classname_2(x):
    return type(x).__name__
classname_1(a), classname_1(i), classname_1(s), classname_2(a), classname_2(i), classname_2(s)

('A', 'int', 'str', 'A', 'int', 'str')

In [2]:
# The Ideal way to create protected members is to use a single underscore '_'
class A:
    def _greet(self):
        print("Hi!")
class B(A):
    def greet(self):
        self._greet()

b = B()
b.greet()

Hi!


In [8]:
# Creating a dict with attribute-like access.
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

params = AttrDict(a='a', b='b')
params.c = 2
vars(params), params.a, params.b, params.c, params['a']

({'a': 'a', 'b': 'b', 'c': 2}, 'a', 'b', 2, 'a')

In [2]:
# Deep copy
# WARNING: copy.deepcopy() is known to be very slow
class A:
    def __init__(self, a):
        self.a = a

# example of deep structure of form
# (list, A, dict, A, int)
v = [A({'a': A(1)})]
u = copy.deepcopy(v)
u[0].a['a'].a = 2
v[0].a['a'].a, u[0].a['a'].a

(1, 2)

In [8]:
# hasattr() still evaluates to true for a attribute that has been set to None,
# and will evaluate to false when using del
class A:
    def __init__(self):
        pass
a = A()
print( hasattr(a, 'a') )
a.a = 1
print( hasattr(a, 'a') )
a.a = None
print( hasattr(a, 'a') )
del a.a
print( hasattr(a, 'a') )

False
True
True
False


In [11]:
# isinstance() to check that an object's class is equal to or a subclass of a class
class A:
    def _greet(self):
        print("Hi!")
class B(A):
    def greet(self):
        self._greet()
a, b = A(), B()
isinstance(a, A), isinstance(a, B), isinstance(b, A), isinstance(b, B)

(True, False, True, True)

In [12]:
# isinstance() to check that an object's class is equal to or a subclass of a class in a tuple.
class A:
    def _greet(self):
        print("Hi!")
class B(A):
    def greet(self):
        self._greet()
a, b = A(), B()
isinstance(a, (A,B,)), isinstance(b, (A,B,))

(True, True)

In [24]:
# Working with slices
l = [0, 1, 2, 3, 4, 5]
s = slice(1, 4)
l[s], s.start, s.stop, s.step, l[slice(None)]

([1, 2, 3], 1, 4, None, [0, 1, 2, 3, 4, 5])

In [105]:
# getting the indices of the longest sequence in a list

def longest_subsequence(l, cond=None):
    """Get the longest subsequence of the list composed of entries
    where cond is true. For example:
    (lambda l, i: l[i], [1, 0, 1, 1, 1, 0, 1]) -> ((slice(2, 5), 3)
    
    Parameters
    ==========
    l : list or np.array
        The list to get subsequence from. 
    cond : function (l, i) -> bool
        To check whether list entries belong to the subsequence.
        If not passed the check using truthiness. 
    
    Returns
    =======
    slice or None
        The slice object to get the subsequence.
    int
        The length of the subsequence.
    """
    if not cond:
        cond = lambda x: x
    n = len(l)
    longest_size  = 0
    longest_begin = None
    longest_end   = None
    curr_end = 0
    while curr_end < n:
        if cond(l[curr_end]):
            curr_begin = curr_end
            while curr_end < n - 1 and cond(l[curr_end+1]):
                curr_end  += 1
            if (curr_end - curr_begin + 1) > longest_size:
                longest_size = curr_end - curr_begin + 1
                longest_begin = curr_begin
                longest_end = curr_end
            curr_end += 1
        curr_end += 1
    if longest_size == 0:
        return None, longest_size
    else:
        return slice(longest_begin, longest_end+1), longest_size

f = lambda x: x
t1 = [1, 0, 1, 1, 1, 0, 1]
t2 = [0, 1, 0, 1, 1, 1, 1, 1]
t3 = [1, 1, 0, 1, 0]
t4 = [0, 1, 0, 1, 0]
t5 = [0, 0]
t6 = [1]
t7 = []

l = t4
r, length = longest_subsequence(l, f)
def d():
    try:
        return l[r]
    except:
        return None
r, length, d()

(slice(1, 2, None), 1, [1])

In [123]:
def longest_sequence_using_split(l, split):
    """Get the longest subsequence of the list after it has been
    split into segments. For example if split is the function
    lambda l, i: l[i + 1] != l[i] + 1 if i < len(l) - 1 else False
    then
    ([1, 2, 3, 2, 3, 4, 5, 1, 2], split) -> ((slice(3, 7), 4)
    as split gives
    [1, 2, 3 | 2, 3, 4, 5 | 1, 2]
    
    Parameters
    ==========
    l : list or np.array
        The list to get subsequence.
    split : function (l, i) -> bool
        To check whether to split the list
        If not passed the check using truthiness.
    
    Returns
    =======
    slice or None
        The slice object to get the subsequence.
    int
        The length of the subsequence.
    """
    n = len(l)
    longest_size  = 0
    longest_begin = None
    longest_end   = None
    curr_begin = 0
    curr_end   = 0
    while curr_end < n:
        if split(l, curr_end) or curr_end == n - 1:
            if (curr_end - curr_begin + 1)  > longest_size:
                longest_size  = curr_end - curr_begin + 1
                longest_begin = curr_begin
                longest_end   = curr_end
            curr_begin = curr_end + 1
            curr_end   = curr_end + 1
        else:
            curr_end  += 1
    if longest_size == 0:
        return None, longest_size
    else:
        return slice(longest_begin, longest_end+1), longest_size

def longest_consecutive_increasing_subsequence(l):
    """Get the longest consecutively increasing subsequence.
    
    Parameters
    ==========
    list of int
        The list to get subsequence.
    
    Returns
    =======
    slice or None
        The slice object to get the subsequence.
    int
        The length of the subsequence.
    """
    def split(l, i):
        try:
            return l[i + 1] != l[i] + 1
        except:
            return False
    return longest_sequence_using_split(l, split)

t1 = [1, 2, 3, 2, 3, 4, 5, 6, 1, 2]
t2 = [3, 2, 3, 4, 5, 6, 7, 1, 2, 1]
t3 = [1, 3, 2, 3, 2, 3, 4, 5]
t4 = [6, 7, 8, 1, 2]
t5 = [1, 3, 5, 7]
t6 = [1]
t7 = [2, 1]
t8 = []

l = t1
r, length = longest_consecutive_increasing_subsequence(l)
def d():
    try:
        return l[r]
    except:
        return None
r, length, d()

(slice(3, 8, None), 5, [2, 3, 4, 5, 6])

In [130]:
def longest_consecutive_decreasing_subsequence(l):
    """Get the longest consecutively decreasing subsequence.
    
    Parameters
    ==========
    list of int
        The list to get subsequence.
    
    Returns
    =======
    slice or None
        The slice object to get the subsequence.
    int
        The length of the subsequence.
    """
    def split(l, i):
        try:
            return l[i + 1] != l[i] - 1
        except:
            return False
    return longest_sequence_using_split(l, split)

t1 = [3, 2, 1, 6, 5, 4, 3, 2, 2, 1]
t2 = [3, 7, 6, 5, 4, 3, 2, 2, 1, 1]
t3 = [1, 3, 3, 2, 5, 4, 3, 2]
t4 = [8, 7, 6, 2, 1]
t5 = [1, 3, 5, 7]
t6 = [1]
t7 = [2, 1]
t8 = []

l = t8
r, length = longest_consecutive_decreasing_subsequence(l)
def d():
    try:
        return l[r]
    except:
        return None
r, length, d()

(None, 0, None)

In [15]:
# locals() returns function's arguments as dict.
def f(a,b, *args,d=None, **kwargs):
    print( locals() )
f(1,2,3,d=4,e=5)

{'a': 1, 'b': 2, 'd': 4, 'args': (3,), 'kwargs': {'e': 5}}


In [28]:
unzip = lambda ll: list(zip(*ll))

def inner_keys_from_nested_dict(d, layers=2):
    ll = []
    vl = [d]
    for layer in range(layers):
        l = []
        f = lambda x: isinstance(x, dict)
        q = collections.deque(util.filter_to_list(f, vl))
        vl = []
        if not q:
            break
        while q:
            d = q.pop()
            keys, values = unzip(d.items())
            vl.append(values)
            l.append(keys)
        l = util.merge_list_of_list(l)
        vl = util.merge_list_of_list(vl)
        ll.append(l)
    return ll

d = {
    'a': {
        'a1': {
            'a11': 11,
            'a12': 12,
        },
        'a2': {
            'a21': 21,
        },
        'a3': 3,
    },
    'b': {
        'a1': 1,
        'b2': {
            'b21': 21,
            'b22': 22,
            'b23': 23,
        },
    },
    'c': {
        'c1': 1,
    },
}

inner_keys_from_nested_dict(d, layers=3)

[['a', 'b', 'c'],
 ['c1', 'a1', 'b2', 'a1', 'a2', 'a3'],
 ['a21', 'a11', 'a12', 'b21', 'b22', 'b23']]

In [13]:
# Using zip, unzip and zip
zip_to_list = lambda *args: list(zip(*args))
unzip = lambda ll: list(zip(*ll))

zip_to_list(* unzip([('a', 1), ('b', 2), ('c', 3), ('d', 4)]) ), \
        unzip( zip_to_list(('a', 'b', 'c', 'd'), (1, 2, 3, 4)) ), \


([('a', 1), ('b', 2), ('c', 3), ('d', 4)],
 [('a', 'b', 'c', 'd'), (1, 2, 3, 4)])

In [42]:
def do_on_nested_dict_of_list(f, dl, *args, **kwargs):
    if isinstance(dl, list):
        f(dl, *args, **kwargs)
    elif isinstance(dl, dict):
        for v in dl.values():
            sort_nested_dict_of_list(v, *args, **kwargs)
    else:
        pass

def sort_nested_dict_of_list(dl, **kwargs):
    def f(l, **kwargs):
        l.sort(**kwargs)
    do_on_nested_dict_of_list(f, dl, **kwargs)

dl = {'a': {'b': [3,1,2]}, 'c': [3,4,1,2], 'd': dict()}
sort_nested_dict_of_list(dl,reverse=True)
dl

{'a': {'b': [3, 2, 1]}, 'c': [4, 3, 2, 1], 'd': {}}

In [3]:
def gen_splits(n):
    """Generator of group indices for (train, val, test) set.
    Only yields n-1 of the possible index combinations"""
    v = util.range_to_list(n)
    for idx in range(0, len(v) - 1):
        yield tuple(v[:idx] + v[idx + 2:]), (idx,), (idx + 1,)

for train, val, test in gen_splits(3):
    print(train, val, test)

(2,) (0,) (1,)
(0,) (1,) (2,)


In [34]:
l = zip(range(10),'aabbbccddd')
s_ = filter(lambda x: x[1] == 'b', l)
l = zip(range(10),'aabbbccddd')
s = filter(lambda x: x[1] == 'b', l)
s = itertools.islice(s, 2)
[*s_], [*s]

([(2, 'b'), (3, 'b'), (4, 'b')], [(2, 'b'), (3, 'b')])

In [3]:
# Cartesian product of iterables
p = itertools.product('ABC', 'ab', range(2))
[*p]

[('A', 'a', 0),
 ('A', 'a', 1),
 ('A', 'b', 0),
 ('A', 'b', 1),
 ('B', 'a', 0),
 ('B', 'a', 1),
 ('B', 'b', 0),
 ('B', 'b', 1),
 ('C', 'a', 0),
 ('C', 'a', 1),
 ('C', 'b', 0),
 ('C', 'b', 1)]